In [ ]:
%load_ext autoreload
%autoreload 2

from gym.environment import MissileEnv, MissileEnvSettings
from agents.proportional_nav import ProportionalNavigationAgent
from models.missile import MissileModel
import models.physics as physics
import numpy as np
import time

from visualization.matplot_viz import MatplotVisualizer, GifSettings
from visualization.plotly_viz import PlotlyVisualizer


settings = MissileEnvSettings()
settings.time_step = 0.1  # Time step for the simulation
settings.realtime = False

target = MissileModel(velocity=np.array([physics.mach_to_ms(3), 0.0,  0.0]), max_acc=100 * 9.81, pos=np.array([0.0, -10_000.0, 20_000.0]))
interceptor = MissileModel(velocity=np.array([0.0, 0.0, physics.mach_to_ms(4.0)]), max_acc=100 * 9.81, pos=np.array([0.0, 0.0, 100.0]))

init_distance = np.linalg.norm(interceptor.pos - target.pos)  # Initial distance between interceptor and target

env = MissileEnv(settings=settings, target=target, interceptor=interceptor)
agent = ProportionalNavigationAgent(max_acc=100*9.81, init_distance=init_distance, n=5.0)

done = False
obs = env.reset()

viz = PlotlyVisualizer()
viz.set_episode_data(env.current_episode)
# viz.render(env.sim_time)

last_time = time.time()
max_render_fps = 10.0

while not done:
    action = agent.step(obs, settings.time_step)  # Get the acceleration command from the agent
    obs, reward, done, _, _ = env.step(action)  # Take a step in the environment

    now = time.time()
    elapsed = now - last_time
    if elapsed >= 1.0 / max_render_fps:
        # viz.render(env.sim_time)  # Render the environment
        last_time = now

viz.playback(env.sim_time, speed=10.0, fps=20.0)  # Playback the simulation

env.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Simulation expired


FigureWidget({
    'data': [{'line': {'color': 'red', 'width': 2},
              'mode': 'lines',
              'name': 'Target',
              'type': 'scatter3d',
              'uid': 'ef3ee317-c1a4-4fbe-9e84-bc3821d5e1fd',
              'x': [102.96],
              'y': [-10000.0],
              'z': [20000.0]},
             {'marker': {'color': 'red', 'size': 6},
              'mode': 'markers',
              'name': 'Target Current Position',
              'type': 'scatter3d',
              'uid': '7f553403-43e7-4981-94be-09d81838323f',
              'x': [102.96],
              'y': [-10000.0],
              'z': [20000.0]},
             {'line': {'color': 'blue', 'width': 2},
              'marker': {'size': 4},
              'mode': 'lines',
              'name': 'Interceptor Agent',
              'type': 'scatter3d',
              'uid': '1c30699e-96e8-49c8-a687-6b9239c203f4',
              'x': [0.0],
              'y': [0.0],
              'z': [237.28]},
             {'mark

# Missile Interceptor Reinforcement Learning

## Proportional Guidance